In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import undetected_chromedriver as uc 
import cfscrape
import time

In [84]:
# Make scraper that bypasses cloudflare
scraper = cfscrape.create_scraper()

# Create undetected chrome instance
options = uc.ChromeOptions()
options.headless = False
driver = uc.Chrome(options=options)

# Start chrome instance
url = 'https://www.edeka24.de/#'
driver.get(url)




C:\Users\mjjyo\AppData\Local\Temp\ipykernel_26336\581803511.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


In [80]:
# Finding the links to all of the pages that have groceries

# Getting the UL 
cookie = driver.find_element(By.XPATH, "//ul[contains(@class, 'nav-list-main') and contains(@class, 'is-level-2')]")

# Getting all the li elements within the UL
li_elements = cookie.find_elements(By.XPATH, ".//li[contains(@class, 'nav-item-main is-level-3')]")

links = []
# Getting href for each li element
for item in li_elements:
    link_holder = item.find_element(By.XPATH, ".//a")
    href = link_holder.get_attribute("href")
    links.append(href)


In [4]:
links

['https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffee-ganze-Bohnen/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffee-gemahlen-Instant/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffeepads/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Kaffeekapseln/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Cappuccino-Kakao/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Filter/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Milch-Sahne-Alternativen/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Tee/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Getraenke/Energydrinks/?force_sid=brua9pqs3chfna0smc4752005j',
 'https://www.edeka24.de/Lebensmittel/Getraenke/Saefte/?force_sid

In [86]:
# Making Dataframe
data = {'category': [], 'sub_category': [], 'product_item': [], 'product_name': [], 'product_price': [], 'product_per_kg/lr': [], 'shipping_cost':[], 'shipping_duration': [], 'date_time': [], 'prod_description': [], 'storage_information': [], 'manf_address': [], 'Ingredients': [], 'Calorif_val_kf': [], 'cal_val_kcal': [], 'fat': [], 'saturated_fat': [], 'carbohydrates': [], 'sugar_carbohydrates': [], 'protein': [], 'salt': [], 'product_url': [], 'product_image': []}

df = pd.DataFrame(data)
df

,category,sub_category,product_item,product_name,product_price,product_per_kg/lr,shipping_cost,shipping_duration,date_time,prod_description,...,Calorif_val_kf,cal_val_kcal,fat,saturated_fat,carbohydrates,sugar_carbohydrates,protein,salt,product_url,product_image


In [6]:
import datetime

In [7]:
# Function for scrolling to the bottom of each page and then selecting all the <a> tags to get links for all grocery products
def get_product_info_links(webdriver, original_page):
    webdriver.get(original_page)

    # This keeps scrolling down and pressing more products until there is no option to anymore
    going = True
    while going is True:
        try:

            # Displaying all food items

            wait = WebDriverWait(driver, 10)
            element = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Weitere Produkte anzeigen')]")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(3.235)
            driver.execute_script("window.scrollBy(0, -325)")



            time.sleep(2.2643)

            # click on the element
            element.click()
            time.sleep(4.5612)


        except:
            going =  False
    
    
    
    # All products have been loaded, now get elements
    
    # Get UL
    all_products_ul = webdriver.find_element(By.ID, 'jqList')
    
    # Get the li
    all_products = webdriver.find_elements(By.XPATH, ".//li[contains(@class, 'js-init')]")
    
    all_products_links = []
    
    # For each li, get the href attached to it
    for item in all_products:
        product_link_holder = item.find_element(By.XPATH, ".//a")
        product_href = product_link_holder.get_attribute("href")
        all_products_links.append(product_href)
    return all_products_links

In [8]:
# Attach all links to one list
every_product = []
for link in links:
    all_products = get_product_info_links(driver, link)
    every_product = every_product + all_products

In [82]:
# Function for getting the details of each product
def get_product_details(webdriver, link):
    webdriver.get(link)
    wait = WebDriverWait(driver, 10)

    try:
        # Top area
        top_classifiers = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "breadcrumb")))

        top_classifiers = top_classifiers.text
        top_classifiers = top_classifiers.split('\n')[-3:]

        category = top_classifiers[0]
        sub_category = top_classifiers[1]
        product_item = top_classifiers[2]

        # Details
        detail_box = webdriver.find_element(By.XPATH, "//div[contains(@class, 'detail-description')]")
        product_name = detail_box.find_element(By.XPATH, ".//h1").text

        product_price_details = detail_box.find_element(By.XPATH, ".//div[contains(@class, 'price')]").text
        product_price_details = product_price_details.split('\n')

        product_price = product_price_details[0]
        product_price_per_weight = product_price_details[1]
        product_shipping_cost = product_price_details[2]
        product_shipping_time = product_price_details[3]

        time_scraped = str(datetime.datetime.now())

        product_description = webdriver.find_element(By.XPATH, ".//div[contains(@class, 'row') and contains(@class, 'article-description-wrap') and contains(@class, 'open')]")
        
        certain_categories_texts = []
        
        
        # Setting up variables
        storage_information = None
        manf_address = None
        Ingredients = None
        Calorif_val_kf = None
        cal_val_kcal = None
        Fat = None
        saturated_fat = None
        Carbohydrates = None
        sugar_carbohydrates = None
        Protein = None
        salt = None
        final_product_description = []
        # Get the categories
        certain_categories = product_description.find_elements(By.XPATH, ".//div[contains(@class, 'listing')]")
        # For each one
        for item in certain_categories:
            
            # Getting certain text items
            item_text = item.text
            if item_text.startswith('Hinweis zur Aufbewahrung'):
                storage_information = item_text
                continue
            elif item_text.startswith('Verantwortliches Lebensmittelunternehmen'):
                manf_address = item_text
                continue
            elif item_text.startswith('Zutatenverzeichnis'):
                Ingredients = item_text
                continue
            else:
                final_product_description.append(item_text)

        product_desc = final_product_description[0]
        
        # Try to get nutrient information
        try:
            nutrient_table = product_description.find_element(By.XPATH, ".//div[contains(@class, 'nutrient-table')]")
            table_text_list = []
            table_text = nutrient_table.find_elements(By.XPATH, ".//tr")
            for item in table_text:
                table_text_list.append(item.text)
        
            
            for i in range(len(table_text_list)):
                info = table_text_list[i]
                info_list = info.split()
                number_item = info_list[-1]
                table_text_list[i] = number_item
            
            Calorif_val_kf = table_text_list[0]
            cal_val_kcal = table_text_list[1]
            Fat = table_text_list[2]
            saturated_fat = table_text_list[3]
            Carbohydrates = table_text_list[4]
            sugar_carbohydrates = table_text_list[5]
            Protein = table_text_list[6]
            salt = table_text_list[7]
            
        except NoSuchElementException:
            pass
        
        
        prod_image = webdriver.find_element(By.XPATH, "//div[contains(@class, 'detail-image')]")
        prod_image_select = prod_image.find_element(By.XPATH, ".//img")
        image_url = prod_image_select.get_attribute('src')
        
        
        return [category, sub_category, product_item, product_name, product_price, product_price_per_weight, product_shipping_cost, product_shipping_time, time_scraped, product_desc, storage_information, manf_address, Ingredients, Calorif_val_kf, cal_val_kcal, Fat, saturated_fat, Carbohydrates, sugar_carbohydrates, Protein, salt, link, image_url]
    except:
        return None

In [87]:
# For each link, use above function and put in dataframe
for i in range(len(every_product)):
    product = every_product[i]
    product_details = get_product_details(driver, product)
    
    if product_details is not None:
        df.loc[i] = product_details
    else:
        continue

In [88]:
df.to_csv(f"C:\\Users\\mjjyo\\OneDrive\\Desktop\\omdena\\edeka_products.csv")

In [89]:
driver.quit()